In [1]:
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import concordance_index_censored
from lifelines import CoxPHFitter
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, auc, roc_curve
from sklearn.model_selection import RandomizedSearchCV

In [3]:
dataset_name = 'TCGA-OV'
data = pd.read_csv("../data/OV/"+dataset_name+"/"+dataset_name+"_data.csv")
data

,case id,PROM1,PROM2,age,OS(d),final_state,stage
0,a2319490-b85d-4219-a1b0-fa1ec432d5c8,1.1163,7.0028,75,2621,1,3
1,a2319490-b85d-4219-a1b0-fa1ec432d5c8,1.1163,13.9069,75,2621,1,3
2,a2319490-b85d-4219-a1b0-fa1ec432d5c8,0.8823,7.0028,75,2621,1,3
3,a2319490-b85d-4219-a1b0-fa1ec432d5c8,0.8823,13.9069,75,2621,1,3
4,42ebd30b-175e-4ece-a806-e55cb7e40e96,0.7281,82.6072,62,949,1,3
...,...,...,...,...,...,...,...
266,c5355491-e1e8-46a4-a05e-bafcaf2e7459,0.7960,14.5609,76,2648,1,3
267,d1976840-35f7-4423-8458-12fb32a52b33,0.2056,31.0689,73,84,1,4
268,d8d13aa4-45d5-4e1a-a6cf-895bdf05e7b2,0.1940,15.0706,63,351,1,4
269,d77ef9cf-f8e6-4ee9-8d4f-1106885f6b06,1.9786,33.7066,67,787,1,3


In [4]:
# Convert the dataset to the structured array format required for sksurv models
data['event_bool'] = data['final_state'].astype(bool)
survival_data = np.array([(row['event_bool'], row['OS(d)']) for _, row in data.iterrows()],
                         dtype=[('event', 'bool'), ('OS(d)', 'float')])

# Features (X) and target (y)
X1 = data[['stage','age']] # clinical data
X2 = data[['PROM1', 'PROM2', 'stage', 'grade']] # all data

In [5]:
X2

array([[-0.47995715, -0.8040769 , -0.31806124,  1.26230518],
       [-0.47995715, -0.6797841 , -0.31806124,  1.26230518],
       [-0.49649209, -0.8040769 , -0.31806124,  1.26230518],
       ...,
       [-0.54512885, -0.65883429,  1.95021762,  0.17488361],
       [-0.41902518, -0.32333501, -0.31806124,  0.53735747],
       [-0.54143322, -0.79806038, -0.31806124, -1.54686723]])

In [7]:
# RSF with cross validation
def RSF_cv(X,y):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    # Variables to store results
    train_cindexes = []
    test_cindexes = []
    aucs = []
    
    param_distributions = {
    'n_estimators': [2,3,4,5,6,7,8,9,10,12,15,18,20,25,30,35,40,45,50,55],
    'max_depth': [2,3,4,5,6,7,8,9,10,12,15,18,20, None],
    'min_samples_split': [2,3,4,5,6,7,8,9,10,11,12],
    'min_samples_leaf': [1,2,3,4,5,6,7,8,9,10]
    }
    
    for fold, (train_index, test_index) in enumerate(kf.split(X), 1):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        rsf = RandomSurvivalForest(random_state=42)
        random_search = RandomizedSearchCV(estimator=rsf, param_distributions=param_distributions, n_iter=10, cv=5, random_state=42, n_jobs=-1)
        random_search.fit(X_train, y_train)
        best_rsf = random_search.best_estimator_
        
        # Evaluate on train and test data
        cindex_train = concordance_index_censored(y_train['event'], y_train['OS(d)'], best_rsf.predict(X_train))[0]
        cindex_test = concordance_index_censored(y_test['event'], y_test['OS(d)'], best_rsf.predict(X_test))[0]

        train_cindexes.append(cindex_train)
        test_cindexes.append(cindex_test)

        # Calculate AUC (using survival probabilities and actual events)
        # Predict survival probabilities for the test set
        y_pred = best_rsf.predict(X_test)

        # Compute AUC score, assuming a binary event outcome
        try:
            auc_score = roc_auc_score(y_test['event'], y_pred)
        except ValueError:  # If AUC cannot be calculated, skip this fold
            auc_score = np.nan
        aucs.append(auc_score)
    #     print(f"Fold {fold} - C-index (Train): {cindex_train:.4f}, C-index (Test): {cindex_test:.4f}, AUC: {auc_score:.4f}")

    # Calculate average C-index and AUC across all folds
    avg_train_cindex = np.mean(train_cindexes)
    avg_test_cindex = np.mean(test_cindexes)
    avg_auc = np.nanmean(aucs)  # Using nanmean to handle potential NaN values

    print(f"Average C-index (Train): {avg_train_cindex:.4f}")
    print(f"Average C-index (Test): {avg_test_cindex:.4f}")
    print(f"Average AUC: {avg_auc:.4f}")

In [8]:
# RSF without cross validation
def RSF(X,y):    
    param_distributions = {
    'n_estimators': [2,3,4,5,6,7,8,9,10,12,15,18,20,25,30,35,40,45,50,55],
    'max_depth': [2,3,4,5,6,7,8,9,10,12,15,18,20, None],
    'min_samples_split': [2,3,4,5,6,7,8,9,10,11,12],
    'min_samples_leaf': [1,2,3,4,5,6,7,8,9,10]
    }
   
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    rsf = RandomSurvivalForest(random_state=42)
    random_search = RandomizedSearchCV(estimator=rsf, param_distributions=param_distributions, n_iter=10, random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    best_rsf = random_search.best_estimator_

    # Evaluate on train and test data
    cindex_train = concordance_index_censored(y_train['event'], y_train['OS(d)'], best_rsf.predict(X_train))[0]
    cindex_test = concordance_index_censored(y_test['event'], y_test['OS(d)'], best_rsf.predict(X_test))[0]

    # Calculate AUC (using survival probabilities and actual events)
    # Predict survival probabilities for the test set
    y_pred = best_rsf.predict(X_test)

    # Compute AUC score, assuming a binary event outcome
    try:
        auc_score = roc_auc_score(y_test['event'], y_pred)
    except ValueError:  # If AUC cannot be calculated, skip this fold
        auc_score = np.nan
#     print(f"Fold {fold} - C-index (Train): {cindex_train:.4f}, C-index (Test): {cindex_test:.4f}, AUC: {auc_score:.4f}")

    print(f"C-index (Train): {cindex_train:.4f}")
    print(f"C-index (Test): {cindex_test:.4f}")
    print(f"AUC: {auc_score:.4f}")

In [9]:
RSF_cv(X1,y)

Average C-index (Train): 0.6547
Average C-index (Test): 0.5500
Average AUC: nan


C:\Users\26620\AppData\Local\Temp\ipykernel_49780\1705079222.py:52: RuntimeWarning: Mean of empty slice
  avg_auc = np.nanmean(aucs)  # Using nanmean to handle potential NaN values


In [10]:
RSF_cv(X2,y)

Average C-index (Train): 0.7260
Average C-index (Test): 0.5211
Average AUC: nan


C:\Users\26620\AppData\Local\Temp\ipykernel_49780\1705079222.py:52: RuntimeWarning: Mean of empty slice
  avg_auc = np.nanmean(aucs)  # Using nanmean to handle potential NaN values


In [11]:
RSF(X1,y)

C-index (Train): 0.7016
C-index (Test): 0.5330
AUC: nan


In [12]:
RSF(X2,y)

C-index (Train): 0.7208
C-index (Test): 0.4794
AUC: nan
